## Module 5 Homework


#### Question 2:
FHV October 2019

Read the October 2019 FHV into a Spark Dataframe with a schema as we did in the lessons.

Repartition the Dataframe to 6 partitions and save it to parquet.

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
import requests

url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz"
response = requests.get(url)

with open("fhv_tripdata_2019-10.csv.gz", "wb") as file:
    file.write(response.content)

print("File downloaded successfully.")

File downloaded successfully.


In [9]:
import pandas as pd

In [10]:
df_fhv_pd = pd.read_csv('fhv_tripdata_2019-10.csv.gz', nrows = 1000)

In [11]:
df_fhv_pd

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264.0,264.0,NaN,B00009
1,B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264.0,264.0,NaN,B00013
2,B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264.0,264.0,NaN,B00014
3,B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264.0,264.0,NaN,B00014
4,B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264.0,264.0,NaN,B00014
...,...,...,...,...,...,...,...
995,B00111,2019-10-01 01:54:57,2019-10-01 03:10:06,264.0,100.0,NaN,B00111
996,B00111,2019-10-01 01:07:06,2019-10-01 01:29:54,264.0,188.0,NaN,B00111
997,B00112,2019-10-01 01:05:36,2019-10-01 01:36:28,264.0,228.0,NaN,B00112
998,B00160,2019-10-01 01:20:00,2019-10-01 01:26:00,264.0,264.0,NaN,B00160


In [12]:
spark.createDataFrame(df_fhv_pd).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [13]:
from pyspark.sql import types

In [14]:
fhv_schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.DoubleType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)]
)


In [15]:
df_fhv = spark.read \
    .option("header", "true") \
    .schema(fhv_schema)\
    .csv('fhv_tripdata_2019-10.csv.gz')

In [16]:
df_fhv.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropOff_datetime', TimestampType(), True), StructField('PUlocationID', IntegerType(), True), StructField('DOlocationID', IntegerType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [18]:
output_path =  'data/pq/fhv/2019/10/'

df_fhv.repartition(6).write.parquet(output_path, mode="Overwrite" )

In [26]:
import os

file_path = 'data/pq/fhv/2019/10/'

def get_avg_parquet_files_size(folder_path):
    total_size = 0
    file_num = 0
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.parquet'):
            file_path = os.path.join(folder_path, file_name)
            total_size += os.path.getsize(file_path)
            file_num += 1
    return (total_size/file_num)/1024/1024

size = round(get_parquet_files_size(file_path),1)

print (f'The average file size of the Parquet files are :{size}mb.')

The average file size of the Parquet files are :6.4mb.


What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

6mb

#### Question 3:

Count records

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

In [27]:
df_fhv.registerTempTable('fhv_data')

C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\sql\dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [40]:
 
spark.sql("""
SELECT *
FROM fhv_data  
LIMIT 10
""").show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [33]:
from pyspark.sql.functions import col


spark.sql("""
SELECT COUNT(*) FROM fhv_data 
WHERE date(pickup_datetime) = '2019-10-15'

""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



62610

In [45]:
from pyspark.sql.functions import  unix_timestamp

df_result = spark.sql("""
SELECT 
    pickup_datetime,
    dropOff_datetime,

    (unix_timestamp(dropOff_datetime) - unix_timestamp(pickup_datetime)) AS duration
FROM
    fhv_data
ORDER BY
    duration DESC
""").show()

+-------------------+-------------------+----------+
|    pickup_datetime|   dropOff_datetime|  duration|
+-------------------+-------------------+----------+
|2019-10-11 18:00:00|2091-10-11 18:30:00|2272149000|
|2019-10-28 09:00:00|2091-10-28 09:30:00|2272149000|
|2019-10-31 23:46:33|2029-11-01 00:13:00| 315620787|
|2019-10-01 21:43:42|2027-10-01 21:45:23| 252460901|
|2019-10-17 14:00:00|2020-10-18 00:00:00|  31658400|
|2019-10-26 21:26:00|2020-10-26 21:36:00|  31623000|
|2019-10-30 12:30:04|2019-12-30 13:02:08|   5272324|
|2019-10-25 07:04:57|2019-12-08 07:54:33|   3804576|
|2019-10-25 07:04:57|2019-12-08 07:21:11|   3802574|
|2019-10-01 13:47:17|2019-11-03 15:20:28|   2856791|
|2019-10-01 07:21:12|2019-11-03 08:44:21|   2856189|
|2019-10-01 13:41:00|2019-11-03 14:58:51|   2855871|
|2019-10-01 18:43:20|2019-11-03 19:43:13|   2854793|
|2019-10-01 18:43:46|2019-11-03 19:43:04|   2854758|
|2019-10-01 07:07:09|2019-11-03 07:58:46|   2854297|
|2019-10-01 14:49:28|2019-11-03 15:38:07|   28

In [46]:
duration_seconds = 2272149000
duration_hours = duration_seconds / 3600
print(duration_hours)

631152.5


631,152.50 Hours

#### Question 5:
User Interface

Spark’s User Interface which shows the application's dashboard runs on which local port?
 
4040 

#### Question 6:
Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark 

Using the zone lookup data and the FHV October 2019 data, what is the name of the LEAST frequent pickup location Zone?

In [47]:

url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"
response = requests.get(url)

with open("taxi_zone_lookup.csv", "wb") as file:
    file.write(response.content)

print("File downloaded successfully.")

File downloaded successfully.


In [50]:
df_zone_pd = pd.read_csv('taxi_zone_lookup.csv', nrows = 1000)

In [51]:
df_zone_pd

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [54]:
 
spark.sql("""
SELECT PUlocationID, COUNT(*) AS count_zone
FROM fhv_data  
GROUP BY PUlocationID
ORDER BY count_zone
LIMIT 10
""").show()

+------------+----------+
|PUlocationID|count_zone|
+------------+----------+
|           2|         1|
|         105|         2|
|         111|         5|
|          30|         8|
|         120|        14|
|          12|        15|
|         207|        23|
|          27|        25|
|         154|        26|
|           8|        29|
+------------+----------+



In [57]:
df_zone = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [60]:
df_zone.registerTempTable('zone_data')

C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\sql\dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [66]:
spark.sql("""
SELECT f.PUlocationID, COUNT(*) AS count_zone, z.Zone 
FROM fhv_data f
JOIN zone_data z ON f.PUlocationID = z.LocationID
GROUP BY f.PUlocationID, z.Zone 
ORDER BY count_zone
LIMIT 10
""").show()

+------------+----------+--------------------+
|PUlocationID|count_zone|                Zone|
+------------+----------+--------------------+
|           2|         1|         Jamaica Bay|
|         105|         2|Governor's Island...|
|         111|         5| Green-Wood Cemetery|
|          30|         8|       Broad Channel|
|         120|        14|     Highbridge Park|
|          12|        15|        Battery Park|
|         207|        23|Saint Michaels Ce...|
|          27|        25|Breezy Point/Fort...|
|         154|        26|Marine Park/Floyd...|
|           8|        29|        Astoria Park|
+------------+----------+--------------------+



The lowest count is Jamaica Bay